In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

### Standard imports

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Important for multiprocessing.
import torch
torch.set_num_threads(1)

### Plots

In [ ]:
from plot import get_3d_subplot_axs
from plot import get_figsize, set_figsize

default_w, default_h = get_figsize()

### Lattice imports

In [ ]:
from collections import OrderedDict
from gridsearch import experiment, load_experiment

# Create NARMA dataset

In [ ]:
import dataset as ds

u_train, y_train = ds.NARMA(sample_len = 2000)
u_test, y_test = ds.NARMA(sample_len = 3000)
dataset = [u_train, y_train, u_test, y_test]
ds.dataset = dataset

# Waxman graph generation

In [ ]:
from matrix import waxman
from plot import scatter_3d

In [ ]:
from matrix import euclidean

# We need names for the distance functions to select them from a pandas
# dataframe later.
euc = euclidean
def inv(x, y): return 1/euclidean(x, y)
def inv_squared(x, y): return 1/euclidean(x, y)**2

### Waxman visualization

In [ ]:
for z_frac in [0.0, 0.5, 1.0]:
    G = waxman(n=200, alpha=1.0, beta=1.0, z_frac=z_frac)
    scatter_3d(G)

### Waxman performance with increasing fraction of 3d nodes

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['hidden_nodes'] = np.arange(20, 90, 10)
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['directed'] = [True, False]
waxman_nrmse_df = experiment(esn_nrmse, params)
waxman_nrmse_df.to_pickle('experiments/waxman_nrmse.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_nrmse_df' not in locals():
    waxman_nrmse_df = load_experiment('experiments/waxman_nrmse.pkl')

df = waxman_nrmse_df.loc[waxman_nrmse_df['directed'] == False]

groupby = ['hidden_nodes', 'z_frac']
axes    = ['hidden_nodes', 'z_frac', 'esn_nrmse']
agg     = ['mean', 'min']
zlim    = (0.5, 1.0)
azim    = 130
title   = 'Increasing fraction of 3d nodes'

plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=title)

There does not seem to be a difference when changing from undirected to directed  
graphs, which I attribute to the fact that I think that both methods are  
saturating the activations of hidden nodes anyway, so it does not matter.  

_Afterthought_ for the above: isn't the spectral radius dealing with that  
anyway? It is probably necessary to actually probe the network to tell for sure.  

The hypothesis that nodes are being saturated are backed up by the fact that the  
error _decreases_ as the networks become smaller, indicating that we are  
reaching a point where nodes may begin to desaturate.  

### Waxman performance with increasing fraction of negative weights

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['z_frac'] = [1.0]
params['hidden_nodes'] = np.arange(20, 90, 10)
params['sign_frac'] = np.arange(0.0, 0.55, 0.05)
params['directed'] = [True, False]
waxman_sign_df = experiment(esn_nrmse, params)
waxman_sign_df.to_pickle('experiments/waxman_sign.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_sign_df' not in locals():
    waxman_sign_df = load_experiment('experiments/waxman_sign.pkl')

undirected_df = waxman_sign_df.loc[waxman_sign_df['directed'] == False]
directed_df = waxman_sign_df.loc[waxman_sign_df['directed'] == True]

groupby = ['hidden_nodes', 'sign_frac']
axes    = ['hidden_nodes', 'sign_frac', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = ['Undirected', 'Directed']
zlim    = (0.3, 1.0)
azim    = 70

for i, df in enumerate([undirected_df, directed_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim,
                    zlim=zlim, title=titles[i])

We see what happens with the undirected (as we think this to be the most  
relevant for physical computing) type of graph in three dimensions, as the  
previous results show little difference between 2 and 3 dimensions.  

There is a noticeable difference between the directed and undirected graphs  
here. Dale et al. investigates this a bit in their paper: «It then becomes clear  
that how weights are struc- tured and directed, controlling information flow,  
has a greater affect on quality of the network. This supports similar results  
using hierarchical networks, where structure and number of parameters also  
significantly impact performance [6].»  

### Distribution of reservoir weights

In [ ]:
from ESN import ESN
from plot import plot_esn_weight_hist

bins = 40
params = {
    'hidden_nodes': 50,
}

plt.title('Reference ESN')
plot_esn_weight_hist(params, n_bins=bins, show=True)

set_figsize(14, 6)

params = {
    'directed': False,
    'w_res_type': 'waxman',
}

hidden_nodes = [20, 50, 50, 50]
dist_functions = [euc, euc, inv, inv_squared]
titles = ['20 nodes euc', '50 nodes euc', '50 nodes inv', '50 nodes inv^2']

for z_frac in [0.0, 1.0]:
    fig, axs = plt.subplots(1, 4)

    params['z_frac'] = z_frac
    plt.suptitle('z_frac:' + str(z_frac))

    for i in range(len(axs)):
        params['hidden_nodes'] = hidden_nodes[i]
        params['dist_function'] = dist_functions[i]

        ax = axs[i]
        ax.set_title(titles[i])
        plot_esn_weight_hist(params, n_bins=bins, ax=ax, show=False)

    plt.show()

set_figsize(default_w, default_h)

We see very clearly that the weight distributions are tightened when we increase  
the number of nodes of the network (changing from leftmost to second to leftmost  
plot), which is caused by the spectral radius scaling to be stricter. However,  
this is just scaling by a scalar value.  

When the weight function moves from d to 1/d to 1/d, we see that we are shifting  
the probability distribution towards many lower weights, with just a few big  
ones, which is as expected.  

### Waxman performance with changing weight/distance function

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['hidden_nodes'] = np.arange(20, 90, 10)
params['dist_function'] = [euc, inv, inv_squared]
waxman_dist_df = experiment(esn_nrmse, params, runs=20)
waxman_dist_df.to_pickle('experiments/waxman_dist.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_dist_df' not in locals():
    waxman_dist_df = load_experiment('experiments/waxman_dist.pkl')

waxman_dist_df['dist_function'] = waxman_dist_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == euc.__name__]
inv_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv_squared.__name__]

groupby = ['hidden_nodes', 'z_frac']
axes    = ['hidden_nodes', 'z_frac', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = ['euc', 'inv', 'inv^2']
labels  = ['euc', 'inv', 'inv^2']
zlim    = (0.3, 1.0)
azim    = 110

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=titles[i])

ax = get_3d_subplot_axs(1)[0]
show = [False, False, True]
for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, zlim=zlim,
                    title='min for all dist functions', agg=['min'], ax=ax, label=labels[i],
                    show=show[i])

It does seem like the different distance functions may all achieve about the  
same performance in terms of error rate on NARMA10. However, the latter ones  
introduce the instability we disappear when we use a minimum aggregation  
instead. The weights in general are _smaller_, as can be seen in the weight  
distribution plots. First thoughts: can this be remedied with input scalings? It  
is clear that _z\_frac_ has little impact on performance, so let's change it to  
input scaling.  

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['input_scaling'] = np.arange(0.1, 2.1, 0.1)
params['hidden_nodes'] = np.arange(20, 90, 10)
params['dist_function'] = [euc, inv, inv_squared]
waxman_is_df = experiment(esn_nrmse, params, runs=20)
waxman_is_df.to_pickle('experiments/waxman_is.pkl')

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['input_scaling'] = np.arange(0.1, 2.1, 0.1)
params['hidden_nodes'] = [80]
params['dist_function'] = [euc, inv, inv_squared]
waxman_is_mc_df = experiment(esn_mc, params, runs=20)
waxman_is_mc_df.to_pickle('experiments/waxman_is_mc.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_is_df' not in locals():
    waxman_is_df = load_experiment('experiments/waxman_is.pkl')

waxman_is_df['dist_function'] = waxman_is_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_is_df.loc[waxman_is_df['dist_function'] == euc.__name__]
inv_df = waxman_is_df.loc[waxman_is_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_is_df.loc[waxman_is_df['dist_function'] == inv_squared.__name__]

groupby = ['hidden_nodes', 'input_scaling']
axes    = ['hidden_nodes', 'input_scaling', 'esn_nrmse']
agg     = ['min']
titles  = ['euc', 'inv', 'inv^2']
zlim    = (0.5, 0.7)
azim    = 20

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=titles[i])

In [ ]:
from plot import plot_df_trisurf

if 'waxman_is_mc_df' not in locals():
    waxman_is_mc_df = load_experiment('experiments/waxman_is_mc.pkl')

waxman_is_mc_df['dist_function'] = waxman_is_mc_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == euc.__name__]
inv_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == inv_squared.__name__]

labels = ['euc', 'inv', 'inv^2']
plt.title('STM (MC) - 80 hidden nodes')
plt.xlabel('Input scaling')
plt.ylabel('STM (MC)')

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    grouped_df = df.groupby(['input_scaling']).mean().reset_index()
    plt.plot(grouped_df['input_scaling'], grouped_df['esn_mc'], label=labels[i])

plt.legend()
plt.show()

Input scaling seems to have a quite big impact on the different distance  
functions. In fact, 1/d^2 has a minimum NRMSE with correct input scaling of 0.5,  
which is quite a lot lower than the original distance function d.  

In the STM plot, we see that 1d^2 consistently has better memory performance  
than the other distance functions, even before changing the input scaling. That  
much. An analysis of the lyapunov spectrum could prove interesting.  

Can we see any patterns if we look the output weights of the network for tests  
of increasing memory capacity?  

In [ ]:
from metric import memory_capacity
from plot import scatter_3d

params = {
    'w_res_type': 'waxman',
    'hidden_nodes': 80,
    'dist_function': inv_squared,
    'input_scaling': 0.1,
}

esn = ESN(**params)
mc = memory_capacity(esn)
print('Memory capacity:', mc)

for i in range(3, 9):
    weights = esn.w_outs[i].abs().data.numpy()
    weights /= weights.max()
    title = 'Hidden weights for STM length: ' + str(i)
    scatter_3d(esn.G, title=title, cols=weights)

I see no concernable pattern, which is perhaps as expected from a fully  
connected network.  

The question remains: what is the difference that makes the inverted functions  
increasingly performant on NARMA/memory? Are the internal node activations  
relevant?  

In [ ]:
from metric import evaluate_esn

# For comparison to a default network.
params = { 'hidden_nodes': 80 }
esn = ESN(**params)
nrmse = evaluate_esn(ds.dataset, esn)
plt.title('Reference NRMSE: ' + str(nrmse))
plt.plot(esn.X)
plt.show()

# With different distance functions.
dist_functions = [euc, inv, inv_squared]
params = {
    'w_res_type': 'waxman',
    'hidden_nodes': 80,
    'input_scaling': 0.1,
}

for f in dist_functions:
    params['dist_function'] = f
    esn = ESN(**params)
    nrmse = evaluate_esn(ds.dataset, esn)

    plt.title(f.__name__ + ' NRMSE: ' + str(nrmse))
    plt.plot(esn.X)
    plt.show()

In [ ]:
from metric import evaluate_esn

# For comparison to a default network.
params = { 'hidden_nodes': 80 }
esn = ESN(**params)
nrmse = evaluate_esn(ds.dataset, esn)
plt.title('Reference NRMSE: ' + str(nrmse))
plt.plot(esn.X)
plt.show()

# With different distance functions.
dist_functions = [euc, inv, inv_squared]
params = {
    'w_res_type': 'waxman',
    'hidden_nodes': 80,
    'input_scaling': 0.1,
    'sign_frac': 0.5,
}

for f in dist_functions:
    params['dist_function'] = f
    esn = ESN(**params)
    nrmse = evaluate_esn(ds.dataset, esn)

    plt.title(f.__name__ + ' NRMSE: ' + str(nrmse))
    plt.plot(esn.X)
    plt.show()

Note: this creates a whole new network every time instead of re-using the  
existing network but changing the weighting scheme, as to try to understand the  
whole picture.  

From running this a few times, I notice that the times when the network simply  
does not work at all, the root cause is that a big majority of the network nodes  
are either *all positive* or *all negative*, there is no balance.  

There is a tendency here: the networks that seem to perform well also seem to  
resemble the default first ESN network in terms of "look", but not so in the  
magnitude of the activations -- which is lowered by the spectral radius. This  
seems to indicate that the initial spectral radius of the waxman networks are  
quite a lot higher than that of default initializations.  

Similar experiments are done twice, but the second time around we introduce  
sign_frac=0.5, which seems to help for the linear distance functions d and 1/d,  
but worsens 1/d^2.  

# Lattice/tiling experiments (sq, rect, hex, tri)

### Plots of lattices

In [ ]:
from ESN import ESN
from plot import plot_lattice

set_figsize(14, 6)

esn_square = ESN(hidden_nodes=25, w_res_type='tetragonal')
esn_hex = ESN(hidden_nodes=25, w_res_type='hexagonal')
esn_tri = ESN(hidden_nodes=25, w_res_type='triangular')
esn_rect = ESN(hidden_nodes=25, w_res_type='rectangular', rect_ratio=2.0)

G_square = esn_square.G
G_hex = esn_hex.G
G_tri = esn_tri.G
G_rect = esn_rect.G

fig, axs = plt.subplots(2, 2)
ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]
plot_lattice(G_square, title='Square', ax=ax1, show=False)
plot_lattice(G_hex, title='Hexagonal', ax=ax2, show=False)
plot_lattice(G_tri, title='Triangular', ax=ax3, show=False)
plot_lattice(G_rect, title='Rectangular', ax=ax4, show=True)

set_figsize(default_w, default_h)

### Growing neighborhoods

In [ ]:
from ESN import ESN
from plot import plot_lattice
from matrix import grow_neighborhoods

set_figsize(14, 16)

esn_sq = ESN(hidden_nodes=25, w_res_type='tetragonal')
G_sq = esn_sq.G
titles = ['"Exponential" growth', 'Default growth']

for n_plt in [0, 1]:
    _G_sq = G_sq.copy()

    fig, axs = plt.subplots(2, 2)
    ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]
    axs = [ax1, ax2, ax3, ax4]

    for i, ax in enumerate(axs):
        plot_lattice(_G_sq, title='Square', ax=ax, neigh_color=True, show=False)

        # The second time around we don't "exponentially" grow the neighborhood,
        # but instead only use the original adjacency matrix to grow the
        # neighborhood `l` times.
        if n_plt == 0:
            grow_neighborhoods(_G_sq, dist_function=inv)
        else:
            _G_sq = G_sq.copy()
            grow_neighborhoods(_G_sq, dist_function=inv, l=i+1)

    plt.suptitle(titles[n_plt])

set_figsize(default_w, default_h)

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
params['grow_neigh'] = list(range(1, 9))
params['dist_function'] = [inv]
lattice_grow_df = experiment(esn_nrmse, params)
lattice_grow_df.to_pickle('experiments/lattice_grow_neigh.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'lattice_grow_df' not in locals():
    lattice_grow_df = load_experiment('experiments/lattice_grow_neigh.pkl')

groupby = ['hidden_nodes', 'grow_neigh']
axes    = ['hidden_nodes', 'grow_neigh', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = ['Square', 'Hexagonal', 'Triangular']
zlim    = (0.5, 0.8)
azim    = -60
elev    = 20

for i, w_res_type in enumerate(['tetragonal', 'hexagonal', 'triangular']):
    df = lattice_grow_df.loc[lattice_grow_df['w_res_type'] == w_res_type]
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, elev=elev,
                    zlim=zlim, title=titles[i])

There is a definite performance penalty with increasing sizes of node  
neighborhood.  

In [ ]:
set_figsize(14, 6)

params = {
    'w_res_type': 'tetragonal',
    'hidden_nodes': 14*14,
    'dist_function': inv,
    'grow_neigh': 0,
    'readout': 'pinv',
}

esn = ESN(**params)
nrmse = evaluate_esn(ds.dataset, esn, plot=True)
print('NRMSE', nrmse )

set_figsize(default_w, default_h)

### Lattice NRMSE

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
nrmse_df = experiment(esn_nrmse, params)
nrmse_df.to_pickle('experiments/lattice_nrmse.pkl')

In [ ]:
if 'nrmse_df' not in locals():
    nrmse_df = load_experiment('experiments/lattice_nrmse.pkl')

grouped_df = nrmse_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_nrmse'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_nrmse'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_nrmse'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('NRMSE')
plt.xlabel('Hidden nodes')

plt.show()

### Lattice STM

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
mc_df = experiment(esn_mc, params)
mc_df.to_pickle('experiments/lattice_mc.pkl')

In [ ]:
if 'mc_df' not in locals():
    mc_df = load_experiment('experiments/lattice_mc.pkl')

grouped_df = mc_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_mc'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_mc'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_mc'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('MC')
plt.xlabel('Hidden nodes')

plt.show()

### Lattice input scaling

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['input_scaling'] = np.insert(np.arange(0.05, 2.05, 0.05), 0, 0.01, axis=0)
params['w_res_type'] = [None, 'tetragonal', 'hexagonal', 'triangular']
waxman_is_df = experiment(esn_mc, params)
is_df.to_pickle('experiments/lattice_input_scaling.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'is_df' not in locals():
    is_df = load_experiment('experiments/lattice_input_scaling.pkl')

reference_esn = is_df.loc[is_df['w_res_type'].isnull()]
tetragonal = is_df.loc[is_df['w_res_type'] == 'tetragonal']
hexagonal = is_df.loc[is_df['w_res_type'] == 'hexagonal']
triangular = is_df.loc[is_df['w_res_type'] == 'triangular']

groupby = ['hidden_nodes', 'input_scaling']
axes    = ['input_scaling', 'hidden_nodes', 'esn_mc']
agg     = ['mean', 'max']
titles  = ['Reference ESN', 'Square', 'Hexagonal', 'Triangular']
zlims   = [(5, 50)] + [(5, 20)]*3
azim    = 70

for i, df in enumerate([reference_esn, tetragonal, hexagonal, triangular]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, zlim=zlims[i], title=titles[i])

### Using a rectangular lattice instead of square

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['w_res_type'] = ['rectangular']
params['rect_ratio'] = np.arange(0.1, 3.1, 0.1)
rect_df = experiment(esn_nrmse, params)
rect_df.to_pickle('experiments/lattice_rect.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'rect_df' not in locals():
    rect_df = load_experiment('experiments/lattice_rect.pkl')

groupby = ['hidden_nodes', 'rect_ratio']
axes    = ['hidden_nodes', 'rect_ratio', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = title = 'Effect of rectangular lattice'
zlim    = (0.5, 0.62)
azim    = 40

plot_df_trisurf(df=rect_df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=title)

### Periodic lattice

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['periodic'] = [True, False]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
periodic_df = experiment(esn_mc, params)
periodic_df.to_pickle('experiments/periodic_lattice.pkl')

In [ ]:
if 'periodic_df' not in locals():
    periodic_df = load_experiment('experiments/periodic_lattice.pkl')

gdf = periodic_df.groupby(['hidden_nodes', 'periodic', 'w_res_type']).mean().reset_index()

from collections import defaultdict
dfs = defaultdict(lambda: [])

plt.title('Periodic lattices')
plt.ylabel('MC')
plt.xlabel('Hidden nodes')
colors = ['green', 'red', 'blue']

for c, l in enumerate(['tetragonal' , 'hexagonal', 'triangular']):
    dfs[l].append(gdf.loc[(gdf['w_res_type'] == l) & (gdf['periodic'] == False)])
    dfs[l].append(gdf.loc[(gdf['w_res_type'] == l) & (gdf['periodic'] == True)])

    for i, p in enumerate(['aperiodic', 'periodic']):
        x = dfs[l][i]['hidden_nodes']
        y = dfs[l][i]['esn_mc']
        label = l + ' ' + p
        linestyle = '-' if i == 0 else '--'
        plt.plot(x, y, label=label, linestyle=linestyle, color=colors[c])

plt.legend(fancybox=False, loc='lower right', bbox_to_anchor=(1.0, 0.0))
plt.show()